# 1. 유료 LLM (Gemini) 코드

In [1]:
# 필요한 패키지 설치
%pip install langchain chromadb google-generativeai langchain-google-genai langchain-chroma faiss-cpu


  Using cached tokenizers-0.20.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.7 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 126.5 MB/s eta 0:00:00a 0:00:01
Using cached tokenizers-0.20.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.0 MB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.21.0
    Uninstalling tokenizers-0.21.0:
      Successfully uninstalled tokenizers-0.21.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
transformers 4.47.1 requires tokenizers<0.22,>=0.21, but you have tokenizers 0.20.3 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [1]:
# 라이브러리 임포트
import os
import json
import pandas as pd
import numpy as np
from collections import Counter

# Langchain 관련
from langchain_google_genai import GoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from langchain_chroma import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA

# 구글 드라이브 데이터 로드를 위한 함수
#from tools.google_drive import ensure_data_files

print("📚 모든 라이브러리 임포트 완료")


📚 모든 라이브러리 임포트 완료


In [2]:
# 데이터 로드
print("📊 데이터 로드 중...")

# Google Drive에서 데이터 다운로드 및 로드
# data_paths = ensure_data_files()

# 각 데이터 파일 로드
diner_df = pd.read_csv('../data/diner.csv')
review_df = pd.read_csv('../data/review.csv')
menu_df = pd.read_csv('../data/dinr_menu.csv')
category_df = pd.read_csv('../data/diner_category_raw.csv')

print(f"✅ 식당 데이터: {diner_df.shape}")
print(f"✅ 리뷰 데이터: {review_df.shape}")
print(f"✅ 메뉴 데이터: {menu_df.shape}")
print(f"✅ 카테고리 데이터: {category_df.shape}")

# 컬럼 정보 확인
print("\n📋 데이터 컬럼 정보:")
print(f"식당: {diner_df.columns.tolist()}")
print(f"카테고리: {category_df.columns.tolist()}")
print(f"메뉴: {menu_df.columns.tolist()}")
print(f"리뷰: {review_df.columns.tolist()}")


📊 데이터 로드 중...


/var/tmp/ipykernel_179573/1148689112.py:8: DtypeWarning: Columns (5,14) have mixed types. Specify dtype option on import or set low_memory=False.
  diner_df = pd.read_csv('../data/diner.csv')


✅ 식당 데이터: (233098, 20)
✅ 리뷰 데이터: (2287474, 7)
✅ 메뉴 데이터: (233044, 2)
✅ 카테고리 데이터: (233044, 6)

📋 데이터 컬럼 정보:
식당: ['diner_idx', 'diner_name', 'diner_tag', 'diner_menu_name', 'diner_menu_price', 'diner_review_cnt', 'diner_review_avg', 'diner_blog_review_cnt', 'diner_review_tags', 'diner_road_address', 'diner_num_address', 'diner_phone', 'diner_lat', 'diner_lon', 'diner_open_time', 'diner_open_time_titles', 'diner_open_time_hours', 'diner_open_time_off_days_title', 'diner_open_time_off_days_hours', 'bayesian_score']
카테고리: ['diner_idx', 'industry_category', 'diner_category_large', 'diner_category_middle', 'diner_category_small', 'diner_category_detail']
메뉴: ['diner_idx', 'diner_menu_name']
리뷰: ['Unnamed: 0', 'diner_idx', 'reviewer_id', 'review_id', 'reviewer_review', 'reviewer_review_date', 'reviewer_review_score']


In [7]:
# Google API 키 설정 (환경변수에서 로드)
# os.environ["GOOGLE_API_KEY"] = "your_google_api_key_here"  # 실제 사용시 설정

# Gemini 모델 설정 (최대 토큰 512)
try:
    llm = GoogleGenerativeAI(
        model="gemini-2.0-flash",
        max_output_tokens=512,
        temperature=0.3
    )
    
    # 임베딩 모델 설정
    embeddings = GoogleGenerativeAIEmbeddings(model="models/gemini-embedding-001")
    
    use_vector_store = True
    print("✅ Gemini 모델 및 임베딩 설정 완료")
    
except Exception as e:
    print(f"⚠️  API 키 없음. 대체 방법 사용: {e}")
    use_vector_store = False
    llm = None
    embeddings = None


✅ Gemini 모델 및 임베딩 설정 완료


In [9]:
diner_df.head()

,diner_idx,diner_name,diner_tag,diner_menu_name,diner_menu_price,diner_review_cnt,diner_review_avg,diner_blog_review_cnt,diner_review_tags,diner_road_address,diner_num_address,diner_phone,diner_lat,diner_lon,diner_open_time,diner_open_time_titles,diner_open_time_hours,diner_open_time_off_days_title,diner_open_time_off_days_hours,bayesian_score
0,2110006612,담원순대,"['제로페이', '모임,회식', '혼밥']","['순대국(일반)', '뼈 해장국(일반)', '감자탕(중)', '모듬수육(대)', ...","[10000, 14000, 16000, 18000, 20000, 34000, 35000]",26,3.9,4.6,"[""('맛', 19)"", ""('가성비', 10)"", ""('친절', 10)"", ""('...",서울 중구 세종대로9길 52,서울 중구 서소문동 120-21,02-778-7210,37.561945,126.973194,NaN,['영업시간'],"['월~금 10:00 ~ 22:00', '토 10:00 ~ 15:00', '월~금 ...",['휴무일'],['일요일'],0.364443
1,1210386151,백년불고기물갈비 서여의도점,"['제로페이', '점심식사']","['산더미소물갈비', '돼지물갈비', '옛날소불고기', '소불고기정식(2인이상)',...","[8000, 10000, 11000, 19000, 20000, 21000]",83,3.4,3.7,"[""('친절', 26)"", ""('맛', 17)"", ""('가성비', 13)"", ""('...",서울 영등포구 은행로 3,서울 영등포구 여의도동 15-24,02-785-8151,37.527506,126.919666,NaN,['영업시간'],"['월~금 11:00 ~ 22:00', '월~금 휴게시간 15:00 ~ 17:00']",['휴무일\n휴무일\n수정 제안\n이동하기'],"['토,일요일, 더보기']",0.364058
2,1357207453,지미존스 강남역점,"['카카오페이', '지미존스', '샌드위치']","['빌리 클럽', '미트볼 파마산', '얼티밋 포커', '스윗 칠리 치킨', '터키...","[6400, 7400, 7900, 8900, 9100]",22,4.6,4.3,"[""('맛', 12)"", ""('친절', 6)"", ""('분위기', 6)"", ""('가성...",서울 강남구 강남대로 358,서울 강남구 역삼동 826-14,02-2051-2024,37.494945,127.029704,NaN,['영업시간'],"['월~금 08:00 ~ 22:00', '토,일 09:00 ~ 22:00']",['휴무일'],['설당일'],0.361879
3,1167589615,특수부돼 면목본점,"['면목역고기집', '이동약자접근', '면목역삼겹살맛집', '면목역맛집', '모임,...","['소금구이모듬', '양념모듬구이', '장미꽃한판세트', '소돼장모듬구이한판', '...","[17000, 30000, 45000, 57000, 88000]",31,4.5,5.0,"[""('친절', 19)"", ""('맛', 15)"", ""('가성비', 12)"", ""('...",서울 중랑구 겸재로 187-8,서울 중랑구 면목동 102-25,02-2207-1160,37.588981,127.088482,NaN,"['영업시간', '공휴일']","['월~목 15:00 ~ 22:00', '금 15:00 ~ 02:00', '토 11...",NaN,[],0.359324
4,2061851602,담원순대 강남직영점,"['카카오페이', '제로페이', '모임,회식', '혼밥']","['순대국', '뼈해장국', '모듬수육 (소)', '감자탕 (소)']","[10000, 20000, 27000]",22,2.7,4.9,"[""('맛', 19)"", ""('친절', 15)"", ""('가성비', 9)"", ""('분...",서울 강남구 테헤란로4길 34,서울 강남구 역삼동 826-30,NaN,37.496146,127.030361,NaN,['영업시간'],"['월~금 09:20 ~ 22:00', '토,일 09:20 ~ 21:00']",NaN,[],0.359048


In [10]:
review_df.tail()

,Unnamed: 0,diner_idx,reviewer_id,review_id,reviewer_review,reviewer_review_date,reviewer_review_score
2287469,2292950,1.429768e+09,3904434829,12070292,"조용한 술집에서 진득하게 한 잔 하고싶어서 방문한\n이자카야였는데, 정말 만족스럽게...",2025-02-06,5.0
2287470,2292951,2.154138e+07,3904572638,12050984,맛있었고 푸짐하고 서비스도 친절한곳입니다.\n주말이라 사람이 많았지만 예약하고 방문...,2025-02-03,5.0
2287471,2292952,1.429768e+09,3904798963,12076245,이자카야 잔잔 맛집이라길래 와봤는데 메뉴 진짜 다양하고 주문한 메뉴 다 맛있어요~\...,2025-02-07,5.0
2287472,2292953,9.800904e+08,3905620833,12066342,"역한 냄새 없이 맛있는 양갈비 먹었어요!\n직원분이 능수능란 정말 친절하시고,\n밑...",2025-02-05,5.0
2287473,2292954,1.461530e+07,3933744715,12209664,기다릴 정도는 아닌 것 같지만 웨이팅이 없다면 바로 들어가 식사할 곳,2025-02-26,4.0


In [12]:
latest_reviews = review_df.sort_values(by='reviewer_review_date', ascending=False)

aggregated_reviews_df = latest_reviews.groupby('diner_idx').agg(
# 리뷰 텍스트를 줄바꿈과 함께 하나의 문자열로 합치기
# NaN 값은 .dropna()로 제거하여 'None'이 합쳐지는 것을 방지
all_reviews=('reviewer_review', lambda x: '\n---\n'.join(x.dropna())),
).reset_index() # groupby 결과는 diner_idx가 인덱스가 되므로, 다시 컬럼으로 만듭니다.
aggregated_reviews_df

,diner_idx,all_reviews
0,3.490700e+04,살면서 가장 맛없는 방어를 먹었습니다.\n한번 씹고 뱉었습니다. 최악입니다.\n대방...
1,4.446200e+04,마라탕\n---\n마라샹궈 여기서 처음 먹었는데 맛있었습니다 다만 꿔바로우는 그냥 ...
2,5.617100e+04,여기서 햄버거 자주 시켜먹는데 오늘 처음 카페라떼시켜먹고 충격이 컸습니다.. \n시...
3,7.064800e+04,집근처에 예쁘게 생긴가게가 생겨서 방문했는데\n정말 너무 맛있고 친절하고 깨끗합니다...
4,1.339150e+05,얼큰 수제비 2번째인데 멸치육수 우린 국물맛 진짜 시원하고 깊은 맛 나요. 다른 메...
...,...,...
163174,2.147362e+09,양념맛이 아주 좋아요! 포장 수령했는데 또갈게요\n---\n양념이 매우 맛있습니다\...
163175,2.147365e+09,알바가 불친절해요. 교육좀 해주세요..\n---\n그냥 쏘쏘 버거킹\n---\n항상...
163176,2.147382e+09,"일일향 매니아인데, 이 지점은 정말 맛이 없습니다.\n미리 튀겨논거 냉동해놨다가 다..."
163177,2.147413e+09,삽겹살 집은 항상 여기만 가요!! 집이랑 가까워서 가보게 되었는데 사장님도 친절하시...


In [15]:
def prepare_restaurant_documents():
    """식당 데이터를 Document 형태로 변환 (강남구 데이터만)"""
    print("🔄 식당 데이터 전처리 시작 (강남구 필터링)...")
    
    documents = []
    
    # 1. 기본 식당 데이터 - 강남구만 필터링
    base_df = diner_df.copy()
    
    # 강남구 필터링: 도로명주소 또는 지번주소에 '강남구'가 포함된 데이터만
    gangnam_mask = (
        base_df['diner_road_address'].fillna('').str.contains('강남구', na=False) |
        base_df['diner_num_address'].fillna('').str.contains('강남구', na=False)
    )
    base_df = base_df[gangnam_mask]
    
    print(f"🔍 강남구 필터링 결과: 전체 {len(diner_df)}개 → 강남구 {len(base_df)}개 식당")
    
    # 2. 카테고리 데이터 병합 (실제 컬럼명 사용)
    category_cols = category_df.columns.tolist()
    print("Category columns:", category_cols)

    # 강남구 식당과 일치하는 카테고리 데이터만 필터링
    gangnam_category_df = category_df[category_df['diner_idx'].isin(base_df['diner_idx'])]
    
    merged_df = pd.merge(base_df, gangnam_category_df[['diner_idx', 'diner_category_large']], 
                        left_on='diner_idx', right_on='diner_idx', how='left')
    merged_df = merged_df.rename(columns={'diner_category_large': 'category'})
    
    # 3. 메뉴 정보 집계 (강남구 식당만)    
    gangnam_menu_df = menu_df[menu_df['diner_idx'].isin(base_df['diner_idx'])]
    menu_grouped = gangnam_menu_df.groupby('diner_idx')['diner_menu_name'].apply(
            lambda x: ', '.join(x.astype(str).unique()[:10])
        ).reset_index()
    menu_grouped.columns = ['diner_idx', 'menu_list']
    merged_df = merged_df.merge(menu_grouped, on='diner_idx', how='left')
    
    # 4. 식당 태그 추가
    merged_df['diner_tag'] = diner_df['diner_tag']
    
    # 5. 리뷰 정보 집계    
    # diner_idx별로 가장 최신 리뷰 선택
    # 날짜 기준으로 내림차순 정렬 후, diner_idx로 그룹화하여 첫 번째 행 (가장 최신)을 선택
    latest_reviews = review_df.sort_values(by='reviewer_review_date', ascending=False)
    aggregated_reviews_df = latest_reviews.groupby('diner_idx').agg(
    # 리뷰 텍스트를 줄바꿈과 함께 하나의 문자열로 합치기
    # NaN 값은 .dropna()로 제거하여 'None'이 합쳐지는 것을 방지
    all_reviews=('reviewer_review', lambda x: '\n---\n'.join(x.dropna())[:1000]),
    ).reset_index() # groupby 결과는 diner_idx가 인덱스가 되므로, 다시 컬럼으로 만듭니다.
    merged_df = pd.merge(merged_df, aggregated_reviews_df, on='diner_idx', how='left')
    
    # 6. 결측값 처리
    merged_df = merged_df.fillna('')
    
    # Document 생성 (타임아웃 방지를 위해 샘플링)
    max_restaurants = 100  # 최대 100개 식당만 사용 (타임아웃 방지)
    # 'diner_review_cnt' 컬럼을 int로 변환
    # errors='coerce': 숫자로 변환할 수 없는 값을 NaN으로 만듭니다.
    merged_df['diner_review_cnt'] = pd.to_numeric(merged_df['diner_review_cnt'], errors='coerce')
    # NaN 값을 0으로 채웁니다.
    merged_df['diner_review_cnt'] = merged_df['diner_review_cnt'].fillna(0)
    # 최종적으로 int 타입으로 변환합니다.
    # fillna(0)을 했기 때문에 NaN은 없고 모두 숫자가 되어 int로 안전하게 변환 가능합니다.
    merged_df['diner_review_cnt'] = merged_df['diner_review_cnt'].astype(int)
    if len(merged_df) > max_restaurants:
        # 리뷰 수가 많은 순으로 정렬하여 상위 100개 선택
        merged_df = merged_df.sort_values('diner_review_cnt', ascending=False).head(max_restaurants)
        print(f"📊 타임아웃 방지: 리뷰 수 높은 상위 {max_restaurants}개 식당만 선택")

    print(f"📄 강남구 {len(merged_df)}개 식당 Document 생성 중...")
    
    for _, row in merged_df.iterrows():
        content = f"""
        식당명: {row.get('diner_name', '')}
        카테고리: {row.get('category', '')}
        태그: {row.get('diner_tag', '')}
        메뉴: {row.get('menu_list', '')}
        평점: {row.get('diner_review_avg', 0)}
        리뷰수: {row.get('diner_review_cnt', 0)}
        지역: 강남구
        주소: {row.get('diner_road_address', '')}
        리뷰:
        {row.get('all_reviews', '') if row.get('all_reviews') else '리뷰 없음'}
        """
        
        metadata = {
            'diner_id': str(row.get('diner_idx', '')),
            'diner_name': str(row.get('diner_name', '')),
            'address': str(row.get('diner_road_address', '')),
            'region': '강남구',
            'category': str(row.get('category', '')),
            'tag': str(row.get('diner_tag', '')),
            'menu': str(row.get('menu_list', '')),
            'reviews': str(row.get('all_reviews', '')),
            'rating': row.get('diner_review_avg', 0),
            'review_count': row.get('diner_review_cnt', 0)
        }
        
        documents.append(Document(page_content=content, metadata=metadata))
    
    print(f"✅ 강남구 {len(documents)}개 Document 생성 완료")
    return documents

# Document 생성
restaurant_docs = prepare_restaurant_documents()


🔄 식당 데이터 전처리 시작 (강남구 필터링)...


🔍 강남구 필터링 결과: 전체 233098개 → 강남구 10555개 식당
Category columns: ['diner_idx', 'industry_category', 'diner_category_large', 'diner_category_middle', 'diner_category_small', 'diner_category_detail']
📊 타임아웃 방지: 리뷰 수 높은 상위 100개 식당만 선택
📄 강남구 100개 식당 Document 생성 중...
✅ 강남구 100개 Document 생성 완료


In [16]:
restaurant_docs

[Document(metadata={'diner_id': '2077289603', 'diner_name': '김태완스시 대치본점', 'address': '서울 강남구 선릉로64길 17', 'region': '강남구', 'category': '일식', 'tag': "['혼밥']", 'menu': "['날치알군함(2ps)', '두가지초밥4', '세트메뉴1', '메밀(여름/4월~9월)', '농어(2ps)', '참치초밥(눈다랑어/10ps)', '광어(2ps)', '세트메뉴2', '두가지초밥8', '열빙어구이(10마리)', '오늘의 초밥 (10p)', '참다랑어뱃살사시미추가시(1ps)', '스페셜롤', '간장새우(10마리)', 'best모듬회(미/2인)', '두가지초밥6', '모밀', '두가지초밥9', '성게알한판', '빵새우튀김', '특선초밥(12ps)', '활생선초밥 (10p)', '모듬회(진/4인)', '수제돈카츠', '눈다랑어(2ps)', '미니사시미(1인)', '타코와사비', '해물야끼우동', '왕새우(2ps)', '대마끼(2ps)', '연어초밥(10ps)', '크림새우', '장어덮밥', '연어샐러드', '참다랑어뱃꼽살(1ps)', 'VIP초밥(14ps)', '해삼내장젓갈추가시(50g)', '두가지초밥2', '대뱃살다짐(2ps)', '계란(2ps)', '연어회덮밥', '참치불초밥(10ps)', 'best스페셜초밥(12ps)', '광어초밥(10ps)', '장어(2ps)', 'best 꽃등심불초밥(2ps)', '성게알(2ps)', '단새우(2ps)', '두가지초밥7', '황새치(2ps)', '활광어회덮밥', '오늘의 초밥(10ps)', '타코와사비(2ps)', '전복(2ps)', '스페셜초밥 (12p)', '감자고로게', '모듬회 (선)', '빵새우튀김(5미)', '우동', '참치회덮밥', '유부(2ps)', '고로케', '꽃등심불초밥(10ps)', '참치회닭밥', '두가지초밥1', '방어(2ps)', '푸짐세트', '싱싱회덮밥', '모듬회 (진)', '도미(2p

In [ ]:
# 리뷰수가 안 맞음
# diner_df['diner_review_cnt'] 와 review_df.groupby('diner_idx')['reviewer_review'].count() 를 비교해보니

In [17]:
# ⚡ 타임아웃 방지를 위한 안전한 벡터 스토어 생성
import time
import random

vector_store = None
qa_chain = None

def vector_store_creation():
    # 1단계: 매우 작은 샘플로 API 연결 테스트
    print("🔬 1단계: API 연결 테스트 (1개 문서)")
    try:
        # 테스트용 작은 문서 생성
        test_doc = Document(
            page_content="강남구 테헤란로 테스트 식당 - 한식, 맛있는 김치찌개",
            metadata={"name": "테스트식당", "address": "강남구"}
        )
        
        test_store = Chroma.from_documents(
            documents=[test_doc],
            embedding=embeddings,
            persist_directory="./chroma_test"
        )
        print("✅ API 연결 성공!")
    
        
        # 작은 청크로 분할
        # text_splitter = RecursiveCharacterTextSplitter(
        #     chunk_size=300,
        #     chunk_overlap=50
        # )
        # split_docs = text_splitter.split_documents(restaurant_docs)
        # print(f"📄 {len(split_docs)}개 작은 청크 생성")
        
        # vector_store = Chroma.from_documents(
        #     documents=split_docs,
        #     embedding=embeddings,
        #     persist_directory="./chroma_db"
        # )
        
        # restaurant_docs: 식당 100개 Document 리스트
        # 이미 각 식당이 Document 하나로 들어있으므로 split 필요 없음
        vector_store = Chroma.from_documents(
            documents=restaurant_docs,
            embedding=embeddings,
            collection_name="restaurants"
        )
        
        return vector_store
        
    except Exception as e:
        print(f"❌ 실패: {str(e)[:80]}...")
        return None

In [18]:
# 벡터 스토어 생성 시도
if use_vector_store:
    print("🚀 타임아웃 방지 모드로 벡터 스토어 생성 시작...")
    
    try:
        vector_store = vector_store_creation()
        
        if vector_store:
            # 성공시 추천 시스템 설정
            recommendation_prompt = PromptTemplate(
                input_variables=["context"],
                template="""
당신은 강남구 전문 식당 추천 AI입니다. 사용자 정보를 바탕으로 강남구의 적합한 식당 5개를 추천해주세요.

사용자 정보:
{context}

다음 형식으로 정확히 5개의 강남구 식당을 추천해주세요:

1. [식당명] - [주소]
   - 카테고리: [카테고리]
   - 추천 이유: [사용자 선호도와 맞는 이유를 간단히 설명]

2. [식당명] - [주소]
   - 카테고리: [카테고리]  
   - 추천 이유: [사용자 선호도와 맞는 이유를 간단히 설명]

3. [식당명] - [주소]
   - 카테고리: [카테고리]
   - 추천 이유: [사용자 선호도와 맞는 이유를 간단히 설명]

4. [식당명] - [주소]
   - 카테고리: [카테고리]
   - 추천 이유: [사용자 선호도와 맞는 이유를 간단히 설명]

5. [식당명] - [주소]
   - 카테고리: [카테고리]
   - 추천 이유: [사용자 선호도와 맞는 이유를 간단히 설명]

모든 답변은 한국어로 작성하고, 강남구 내에서만 추천해주세요.
                """
            )
            
            qa_chain = RetrievalQA.from_chain_type(
                llm=llm,
                chain_type="stuff",
                retriever=vector_store.as_retriever(search_kwargs={"k": 5}),
                chain_type_kwargs={"prompt": recommendation_prompt},
                input_key="context",  # <--- Change this line!
                return_source_documents=True  # Often useful for debugging, can remove later
            )
            
            print("✅ 타임아웃 방지 벡터 스토어 시스템 설정 완료!")
            use_vector_store = True
        else:
            raise Exception("벡터 스토어 생성 실패")
            
    except Exception as e:
        print(f"⚠️  모든 시도 실패: {str(e)[:100]}...")
        print("🔄 100% 안전한 키워드 매칭 시스템으로 전환")
        vector_store = None
        qa_chain = None
        use_vector_store = False
        
else:
    print("⚠️  API 키 없음. 대체 추천 시스템 사용")

# 최종 시스템 상태
if not use_vector_store:
    print("✅ 키워드 매칭 기반 추천 시스템 활성화")
    print("🛡️ 이 시스템은 타임아웃 에러가 절대 발생하지 않습니다!")
else:
    print("🎉 벡터 스토어 시스템 활성화 성공!")

print("🚀 추천 시스템 준비 완료 - 다음 셀을 실행하세요!")


🚀 타임아웃 방지 모드로 벡터 스토어 생성 시작...
🔬 1단계: API 연결 테스트 (1개 문서)


✅ API 연결 성공!
✅ 타임아웃 방지 벡터 스토어 시스템 설정 완료!
🎉 벡터 스토어 시스템 활성화 성공!
🚀 추천 시스템 준비 완료 - 다음 셀을 실행하세요!


In [19]:
# retriever 잘되는지 확인
retriever = vector_store.as_retriever(search_kwargs={"k": 5})
# 예시: 특정 사용자 쿼리에 대해 retriever 결과 확인
query = "강남구에서 혼밥하기 좋은 스시집 추천해줘"
docs = retriever.get_relevant_documents(query)
print(f"검색된 문서 수: {len(docs)}\n")
for i, doc in enumerate(docs, start=1):
    print(f"[{i}] {doc.metadata['diner_name']} ({doc.metadata['category']})")
    print(f"리뷰 예시: {doc.metadata['reviews'][:300]}...\n")

검색된 문서 수: 5

[1] 갓덴스시 강남점 (일식)
리뷰 예시: 맛나구 친절
---
불러서 없으면 취소시킨다고 써 놓고 3번 4번부르고 나중에 오면 들여보내고 앞에서 기다리는 사람은 호구인가요
다음엔 대기등록하고 1시간후에 올겁니다.
---
맛은 좀 아쉬운데 서비스랑 시설이 좋음
---
스시를 시킬수도 있어서 맛있었어요!
샌각보다 저렴해서 국
---
짧은 대기 시간, 친절한 서비스.
---
회전초밥집인데 퀄리티 좋고 맛있고 혼밥하기도 괜찮습니다. 단점은 비싸서 7접시 먹었는데 4만원 나왔네요
---
1시간 기다려서 먹었는데 기다릴만한가에 대한 고민중..
---
불친절 비위생
---
스시 엄청 좋...

[2] 김태완스시 대치본점 (일식)
리뷰 예시: 주말에 와이프랑 스시먹으러 왔는데 너무너무 맛있어요!!!:) 맥주도 시원해요
---
영업정지중인데 리뷰조작까지 하는곳이라 못참겠닼ㅋㅋㅋㅋ
---
이 동네에서 제일 맛있는 초밥집입니다~ 생선 양도 많고 퀄리티도 아주 훌륭해요! 스시가 신선하고 맛도 진하고 깔끔합니다 ㅎㅎ
---
초밥 세트에 미니 우동까지 가성비 좋고, 단무지에 유자맛 나는거 최고다…!!!
---
라멘 정식 맛있어요
---
와이프와 커플세트 주문했습니다
적당한 가격에 푸짐하고 맛있는 메뉴! 만족스럽네요
---
초밥 구성도 좋고 하나하나 너무 맛있었습니다! 매장손님 말고...

[3] 은행골 신사점 (일식)
리뷰 예시: 포장 전화로 안 되고 방문해서만 가능함
맛은 정말 좋은데 알바생 불친절 ㅠㅠ 
매장 식사시에는 서비스 많은데 포장은 서비스 없음. 
근데 맛은 좋아서 별 4
---
굳친절정도
---
이쪽 근무지 다닐때 
못해도 한 30여번은 간듯 ㅋ
물리면 가끔 큰맘먹고 김수사 가고
최애초밥집
---
연어초밥 도로초밥 맛집
---
그때 그시절 여친과 맛난거 먹자고 날잡고 은행골을 가던 때가있었다 정말 순수하던 우리.. 잘지네니?
---
갈 때마다 기분 좋아지는 식당입니다🥹
맛, 친절, 서비스 최고에요.
초밥도 너무 맛있고
신사에서 밥 먹을 때마다 .

/var/tmp/ipykernel_57014/2470572755.py:5: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  docs = retriever.get_relevant_documents(query)


In [20]:
queries = [
    "강남구 혼밥하기 좋은 곳",
    "회식하기 좋은 식당",
    "양식 레스토랑 추천"
]
for q in queries:
    print("="*50)
    print("Query:", q)
    docs = retriever.get_relevant_documents(q)
    for i, doc in enumerate(docs, start=1):
        print(f"  [{i}] {doc.metadata['diner_name']} ({doc.metadata['category']}")
        print(f"리뷰 예시: {doc.metadata['reviews'][:300]}...\n")

Query: 강남구 혼밥하기 좋은 곳
  [1] 농민백암순대 강남직영점 (한식
리뷰 예시: 국물이 조미료맛보단 깊이가 있는 사골국이네요 순대도 맛있어요
---
인생최고의국밥
농백순을 알기 전과 후로 국밥에 대한 제 가치관이 바뀌었습니다 꼭 먹어보시길 바랍니다.
---
최고의 순대국집
---
가게 깔끔하고 양 왕 많음 맛있음
---
맛과 품질보다는 입지가 좋아요
---
12시 도착했는데 이미 사람들 많이 기다리고 있었어요. 도착하면 바로 문옆에 붙어있는 대기명단에 이름 쓰세요. 대기자가 많아도 회전이 빨라서 앞에 14팀 있었는데 한 20분 정도만 기다렸어요. 
순대 정식 시키면 사진에 보이듯 순대국이랑 순대 네점과 수육 몇...

  [2] 마녀주방 강남점 (양식
리뷰 예시: 쏘쏘
---
저렴한 맛에 갈 만 한 것 같음. 10~20년 전 할로윈 느낌
---
진짜 너무 맛있어요 1달에 한 번은 꼭 찾아옴요
부채살 크림 리조또랑 게살 파스타 꼭 드세요
---
마녀? 할로윈? 컨셉에 맞춰서 잘 꾸며있어요 
고르곤졸라랑 로제파스타 먹었는데 음료 없으면 
중간부터 좀 느끼하네요 : ) 그래도 먹을만 합니다
---
무난무난. 토마호크 고기가 약간 질김. 소스 취향 갈릴 듯. 토마토 파스타는 괜찮음! 분위기 독특함 ㅋㅋㅋㅋㅋ
---
술 마실때 한 번 가볼만한 곳 !
컨셉과 분위기에 취할수도 있음
---
맛있게 잘 먹...

  [3] 구구당 (퓨전요리
리뷰 예시: 알고있는 무난한 맛인데 가게됨
파스타 굳
---
맛 별로, 직원은 불친절
---
지하 홀보는 중국인 알바 일하기 겁나 싫어보이던데 그럴거면 그냥 집에나 있어라 왜 하기싫은 티 팍팍 내면서 밥맛 떨어지게 하나?
---
솔직히 무난한데 알고있는 맛잇는 무난함이라 또 갈 듯
---
전 꿔바로우는 제 스탈 아니었고, 마라 파스타, 볶음밥 다 마라 맛 나서 좋았어요. 
지난 번 방문에는 무슨 치킨 텐더 있는 누들 시켰던 것 같은데
그것보단 나은듯..
재방문 의사는 내 의지로는 안 가고 타인이 가자하면 뭐.. 그래하고 갈 것

  [1] 호보식당 (한식
리뷰 예시: 후기장난질 당했음.
직원들은 자기내들끼리 껴안고 연애질하고 주차비 지원안되서 만원 내라그러고 리뷰보고갔더니 맛은 수준이하고
---
주차 가능~!
삼겹살 목살 맛있게 잘 먹었습니다ㅎㅎ
---
외부 인테리어 보고 비쌀줄 알았는데 가격도 저렴하고 정말 맛있는 저녁 한끼였습니다. 이 찰살 먹을려고 예약까지 한다는데 저는 예약안하고 단번에 먹었네요^^
---
제발 카카오맵에는 후기 장난치지 맙시다…
---
댓글알바 쓰시는 비용으로 가게와 맛에 더 투자하시는게 양심상 낫지 않을까요?
---
꼭 가야할가게인가보군..
---
된장찌개, 볶음밥, ...

  [2] 광화문석갈비 코엑스점 (한식
리뷰 예시: 친구들 초대 대접하기 좋아요 분위기있어요 맛도 좋고요
---
맛있고 친절, 맛, 분위기 깡패 좋습니다~!
---
1인상으로 고기먹고싶은때 점심 회식장소로 괜찮습니다. 갈비찜 고기 야들합니다. 양이 좀 아쉽고 국-어묵국 이나 반찬-미역줄기 이 딱히 어울리진 않아서 아쉽네요.
---
코엑스에서 아이랑 외식하기 좋아요
맛있어요
---
바쁘신대도 직원불이 다들 친절하세요. 크리스마스에 이용해서 그런지 음식나오는데 시간이 오래 걸리네요. 오래걸린다고 미리 말씀해 주시면 더 좋았을거 같네요.
---
너무 맛있고, 분위기가 좋은 식당입니다~
-...

  [3] 정식당 (퓨전요리
리뷰 예시: 뉴욕 정식당의 3스타 소식을 듣고 2년만에 다시 찾았어요~
2년전보다 더 맛있어졌네요~^^
발렌타인데이라 특별히 주신 디저트도 너무 맛있었어요~ㅎㅎ
---
그래도 역시 정식당
다만 김밥은 기대 너무 많이하지 않는게... 특히 참치는 시키지말길 차라리 기본이나 돈 더주고 우니를 ㅠㅠ
---
개인적으로 메뉴 구성에 트러플이 과하게 많았던것 같았음. 문어 요리가 약간 질겼음.
---
여러가지 맛의 조합이 신선하고, 서비스의 퀄리티가 훌륭한 곳입니다. 퓨전 한식의 정수.
---
정말 너무 맛있었다🤍 반찬이 제일 맛있긴 함
---
잘 먹고 ...

  [4] 뱃고동 (

In [21]:
# 서울 지역 기반 4개 사용자 쿼리
sample_queries = [
    {
        "query_id": 1,
        "user_info": """
        위치 정보: 서울 강남구, 주로 회사 근처에서 식사
        기본 정보: 30대 남성, 직장인, 점심시간에 빠른 식사 선호
        취향 정보: 매운맛 좋아함, 해산물 알러지 있음
        음식점 평가: 
        - 김밥천국: 가성비 좋고 빠름 (4점)
        - 맥도날드: 편하지만 단조로움 (3점)
        - 한정식집: 맛있지만 시간이 오래 걸림 (4점)
        """
    },
    {
        "query_id": 2, 
        "user_info": """
        위치 정보: 서울 이태원, 트렌디한 분위기 선호
        기본 정보: 20대 여성, 대학생, 친구들과 함께 식사 즐김
        취향 정보: 매운맛 못 먹음, 유제품 알러지 있음, 인스타그램 사진 찍기 좋은 곳 선호
        음식점 평가:
        - 스타벅스: 분위기 좋고 사진 예쁘게 나옴 (5점)
        - 이태원 양식당: 이국적이지만 비쌈 (3점)
        - 예쁜 카페: 달콤하고 예쁘지만 가격 부담 (4점)
        """
    },
    {
        "query_id": 3,
        "user_info": """
        위치 정보: 서울 성수동, 힙한 지역에서 식사 즐김
        기본 정보: 50대 남성, 자영업자, 든든한 한 끼 식사 중요시
        취향 정보: 전통 한식 선호, 매운맛 보통, 특별한 알러지 없음
        음식점 평가:
        - 성수동 맛집: 진짜 집밥 같고 푸짐함 (5점)
        - 족발집: 양 많고 맛있지만 기름져서 부담 (3점)
        - 삼겹살집: 고기 질 좋고 밑반찬 풍성 (4점)
        """
    },
    {
        "query_id": 4,
        "user_info": """
        위치 정보: 서울 홍대, 젊은 분위기 좋아함
        기본 정보: 40대 여성, 주부, 가족과 함께 식사, 건강한 음식 선호
        취향 정보: 담백한 맛 선호, 견과류 알러지 있음, 가족 친화적인 곳 선호
        음식점 평가:
        - 홍대 브런치카페: 분위기 좋지만 시끄러움 (3점)
        - 건강한 샐러드집: 신선하고 담백함 (5점)  
        - 가족 레스토랑: 아이들과 가기 좋음 (4점)
        """
    }
]

print("📝 4개 사용자 쿼리 정의 완료")
for i, query in enumerate(sample_queries, 1):
    location = [line for line in query["user_info"].split('\n') if '위치 정보' in line][0]
    print(f"   쿼리 {i}: {location.strip()}")


📝 4개 사용자 쿼리 정의 완료
   쿼리 1: 위치 정보: 서울 강남구, 주로 회사 근처에서 식사
   쿼리 2: 위치 정보: 서울 이태원, 트렌디한 분위기 선호
   쿼리 3: 위치 정보: 서울 성수동, 힙한 지역에서 식사 즐김
   쿼리 4: 위치 정보: 서울 홍대, 젊은 분위기 좋아함


In [22]:
def get_restaurant_recommendations(query_data):
    """사용자 정보를 기반으로 식당을 추천"""
    try:
        if use_vector_store and qa_chain and vector_store is not None:
            # 벡터 스토어 + Gemini 사용
            print(f"   🔍 벡터 검색 사용 중...")
            print(query_data['user_info'])
            response = qa_chain.invoke({"context": query_data['user_info']})
            ai_response = response["result"] if isinstance(response, dict) else str(response)
        else:
            # 대체 방법 사용 (키워드 매칭)
            print(f"   🔧 키워드 매칭 시스템 사용 중...")
            #ai_response = recommend_restaurants_seoul(query_data["user_info"], restaurant_docs)
            ai_response = None
        
        return {
            "query_id": query_data["query_id"],
            "user_info": query_data["user_info"],
            "ai_response": ai_response
        }
    except Exception as e:
        print(f"   ❌ 에러 발생, 대체 시스템으로 전환: {str(e)[:50]}...")
        # 에러 발생시 무조건 키워드 매칭 시스템 사용
        #ai_response = recommend_restaurants_seoul(query_data["user_info"], restaurant_docs)
        ai_response = None
        return {
            "query_id": query_data["query_id"],
            "user_info": query_data["user_info"],
            "ai_response": ai_response
        }

# 4개 쿼리 실행
print("🔄 4개 쿼리에 대한 식당 추천 실행 중...")
results = []

seoul_areas = ["강남구", "이태원", "성수동", "홍대"]

for i, query in enumerate(sample_queries):
    print(f"\n📍 쿼리 {i+1} ({seoul_areas[i]}) 처리 중...")
    result = get_restaurant_recommendations(query)
    results.append(result)
    print(f"✅ 쿼리 {i+1} 완료")

print("\n🎉 모든 쿼리 처리 완료!")


🔄 4개 쿼리에 대한 식당 추천 실행 중...

📍 쿼리 1 (강남구) 처리 중...
   🔍 벡터 검색 사용 중...

        위치 정보: 서울 강남구, 주로 회사 근처에서 식사
        기본 정보: 30대 남성, 직장인, 점심시간에 빠른 식사 선호
        취향 정보: 매운맛 좋아함, 해산물 알러지 있음
        음식점 평가: 
        - 김밥천국: 가성비 좋고 빠름 (4점)
        - 맥도날드: 편하지만 단조로움 (3점)
        - 한정식집: 맛있지만 시간이 오래 걸림 (4점)
        


✅ 쿼리 1 완료

📍 쿼리 2 (이태원) 처리 중...
   🔍 벡터 검색 사용 중...

        위치 정보: 서울 이태원, 트렌디한 분위기 선호
        기본 정보: 20대 여성, 대학생, 친구들과 함께 식사 즐김
        취향 정보: 매운맛 못 먹음, 유제품 알러지 있음, 인스타그램 사진 찍기 좋은 곳 선호
        음식점 평가:
        - 스타벅스: 분위기 좋고 사진 예쁘게 나옴 (5점)
        - 이태원 양식당: 이국적이지만 비쌈 (3점)
        - 예쁜 카페: 달콤하고 예쁘지만 가격 부담 (4점)
        
✅ 쿼리 2 완료

📍 쿼리 3 (성수동) 처리 중...
   🔍 벡터 검색 사용 중...

        위치 정보: 서울 성수동, 힙한 지역에서 식사 즐김
        기본 정보: 50대 남성, 자영업자, 든든한 한 끼 식사 중요시
        취향 정보: 전통 한식 선호, 매운맛 보통, 특별한 알러지 없음
        음식점 평가:
        - 성수동 맛집: 진짜 집밥 같고 푸짐함 (5점)
        - 족발집: 양 많고 맛있지만 기름져서 부담 (3점)
        - 삼겹살집: 고기 질 좋고 밑반찬 풍성 (4점)
        
✅ 쿼리 3 완료

📍 쿼리 4 (홍대) 처리 중...
   🔍 벡터 검색 사용 중...

        위치 정보: 서울 홍대, 젊은 분위기 좋아함
        기본 정보: 40대 여성, 주부, 가족과 함께 식사, 건강한 음식 선호
        취향 정보: 담백한 맛 선호, 견과류 알러지 있음, 가족 친화적인 곳 선호
        음식점 평가:
        - 홍대 브런치카페: 분위기 좋지만 시끄러움 (3점)
        - 건강한 샐러드집: 신선하고 담백함 (5점)  
        - 가족 레스토랑: 아이들과 가기 좋음 (4점)
        
✅ 쿼리 4 완료

🎉 모든 쿼리 처리 완료!


In [23]:
# 최종 JSON 결과 생성
final_output = {
    "restaurant_recommendation_results": results,
    "summary": {
        "total_queries": len(results),
        "region": "서울시 강남구 (테헤란로, 역삼동, 삼성동, 청담동)",
        "system_info": {
            "llm_model": "gemini-2.0-flash",
            "max_tokens": 512,
            "vector_store": "ChromaDB" if use_vector_store else "keyword_matching",
            "embedding_model": "gemini-embedding-001" if use_vector_store else "none"
            # "regions_covered": ["강남구 테헤란로", "강남구 역삼동", "강남구 삼성동", "강남구 청담동"]
        }
    }
}

# JSON 형태로 출력
print("="*80)
print("🍽️ 강남구 전용 식당 추천 시스템 최종 결과 (JSON)")
print("="*80)

json_output = json.dumps(final_output, ensure_ascii=False, indent=2)
print(json_output)


🍽️ 강남구 전용 식당 추천 시스템 최종 결과 (JSON)
{
  "restaurant_recommendation_results": [
    {
      "query_id": 1,
      "user_info": "\n        위치 정보: 서울 강남구, 주로 회사 근처에서 식사\n        기본 정보: 30대 남성, 직장인, 점심시간에 빠른 식사 선호\n        취향 정보: 매운맛 좋아함, 해산물 알러지 있음\n        음식점 평가: \n        - 김밥천국: 가성비 좋고 빠름 (4점)\n        - 맥도날드: 편하지만 단조로움 (3점)\n        - 한정식집: 맛있지만 시간이 오래 걸림 (4점)\n        ",
      "ai_response": "분석 결과, 사용자님은 다음과 같은 특징을 가진 식당을 선호하는 것으로 보입니다.\n\n*   **한식 선호:** 무월식탁, 능라도, 뱅뱅막국수 등 한식 경험이 많음\n*   **다양한 분위기:** 노포부터 깔끔한 식당까지 다양한 분위기를 즐김\n*   **가성비 중시:** 마녀주방처럼 가성비 좋은 곳도 방문\n*   **특색 있는 메뉴:** 과일 막걸리, 퓨전 한식 등 특색 있는 메뉴에 관심\n*   **혼밥 가능:** 혼밥 경험이 있음\n\n이러한 정보를 바탕으로 강남구 내 식당 5곳을 추천해 드립니다.\n\n1.  **강남고속버스터미널 지하상가** - 서울 서초구 신반포로 194 강남고속버스터미널 지하\n    *   카테고리: 한식 (분식, 식사류)\n    *   추천 이유: 저렴한 가격에 다양한 한식 메뉴를 즐길 수 있습니다. 혼밥하기에도 부담 없고, 노포 분위기를 느낄 수 있는 곳입니다.\n\n2.  **영동원** - 서울 강남구 논현로151길 11\n    *   카테고리: 한식 (고기)\n    *   추천 이유: 가성비 좋은 가격에 푸짐한 고기를 즐길 수 있습니다. 낮술을 즐기기에도 좋고, 회식이나 모임 장소로도 적합합니다.\n\n3.  **삼성동 

In [24]:
# 요약 결과 출력
print("\n" + "="*80)
print("📊 결과 요약")
print("="*80)

for i, result in enumerate(results):
    # 사용자 정보 요약
    user_lines = result["user_info"].strip().split('\n')
    for line in user_lines:
        if line.strip():
            print(f"  {line.strip()}")
    
    print(f"\n🍴 추천 결과 :")
    response_lines = result["ai_response"].split('\n')
    for j, line in enumerate(response_lines):
        if line.strip():
            print(f"  {line.strip()}")
    print("  ...")
    print("-" * 60)


📊 결과 요약
  위치 정보: 서울 강남구, 주로 회사 근처에서 식사
  기본 정보: 30대 남성, 직장인, 점심시간에 빠른 식사 선호
  취향 정보: 매운맛 좋아함, 해산물 알러지 있음
  음식점 평가:
  - 김밥천국: 가성비 좋고 빠름 (4점)
  - 맥도날드: 편하지만 단조로움 (3점)
  - 한정식집: 맛있지만 시간이 오래 걸림 (4점)

🍴 추천 결과 :
  분석 결과, 사용자님은 다음과 같은 특징을 가진 식당을 선호하는 것으로 보입니다.
  *   **한식 선호:** 무월식탁, 능라도, 뱅뱅막국수 등 한식 경험이 많음
  *   **다양한 분위기:** 노포부터 깔끔한 식당까지 다양한 분위기를 즐김
  *   **가성비 중시:** 마녀주방처럼 가성비 좋은 곳도 방문
  *   **특색 있는 메뉴:** 과일 막걸리, 퓨전 한식 등 특색 있는 메뉴에 관심
  *   **혼밥 가능:** 혼밥 경험이 있음
  이러한 정보를 바탕으로 강남구 내 식당 5곳을 추천해 드립니다.
  1.  **강남고속버스터미널 지하상가** - 서울 서초구 신반포로 194 강남고속버스터미널 지하
  *   카테고리: 한식 (분식, 식사류)
  *   추천 이유: 저렴한 가격에 다양한 한식 메뉴를 즐길 수 있습니다. 혼밥하기에도 부담 없고, 노포 분위기를 느낄 수 있는 곳입니다.
  2.  **영동원** - 서울 강남구 논현로151길 11
  *   카테고리: 한식 (고기)
  *   추천 이유: 가성비 좋은 가격에 푸짐한 고기를 즐길 수 있습니다. 낮술을 즐기기에도 좋고, 회식이나 모임 장소로도 적합합니다.
  3.  **삼성동 코엑스 푸드코트** - 서울 강남구 영동대로 513
  *   카테고리: 한식, 양식, 일식 등
  *   추천 이유: 다양한 종류의 음식을 한 곳에서 즐길 수 있습니다. 혼밥하기에도 좋고, 여러 명이 함께 가서 각자 원하는 메뉴를 선택할 수 있습니다.
  4.  **역삼동 김치찌개** - 서울 강남구 역삼로17길 51
  *   카테고리: 한식 (찌개)

# 2. 무료 LLM (Qwen, bge-m3) 코드

In [3]:
%pip install "transformers==4.43.3" "sentence-transformers==2.7.0" "accelerate" "faiss-cpu"

Defaulting to user installation because normal site-packages is not writeable
  Using cached transformers-4.43.3-py3-none-any.whl.metadata (43 kB)
  Using cached faiss_cpu-1.12.0-cp311-cp311-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl.metadata (5.1 kB)
  Using cached tokenizers-0.19.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.7 kB)
Using cached transformers-4.43.3-py3-none-any.whl (9.4 MB)
Using cached tokenizers-0.19.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.6 MB)
Using cached faiss_cpu-1.12.0-cp311-cp311-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl (31.4 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4/4 [sentence-transformers]ence-transformers]
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
flagembedding 1.3.5 requires transformers>=4.44.2, but you have transformers 4.43.3 which is incompatible.
vll

In [ ]:
# pip install transformers==4.43.3 accelerate sentence-transformers faiss-cpu
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch, faiss

# 1) 샘플 맛집 데이터 (실제에선 DB/크롤링/지도 API 연동)
docs = [
  {"name":"을지로 골뱅이집","tags":["안주맛집","매콤","소주"],"area":"을지로","price":"₩₩","desc":"골뱅이무침, 오징어볶음이 유명한 노포 느낌"},
  {"name":"성수 파스타","tags":["파스타","데이트","와인"],"area":"성수","price":"₩₩₩","desc":"생면 파스타와 내추럴와인, 분위기 좋음"},
  {"name":"망원 김밥","tags":["가성비","포장","한식"],"area":"망원","price":"₩","desc":"참치마요, 진미채 김밥이 인기. 포장 대기 짧음"},
  {"name":"연남 냉면집","tags":["여름","시원한","메밀"],"area":"연남","price":"₩₩","desc":"비빔/물냉 모두 탄탄, 육향 깔끔"},
  {"name":"홍대 비건 카페","tags":["비건","디저트","커피"],"area":"홍대","price":"₩₩","desc":"두유 라떼와 통밀 케익, 아늑한 좌석"}
]

# 2) 한국어 임베딩(BGE-M3 계열)로 문서 인덱스 생성
emb_model = SentenceTransformer("upskyy/bge-m3-korean")  # or "BAAI/bge-m3"
corpus = [f"{d['name']} | {d['area']} | {', '.join(d['tags'])} :: {d['desc']}" for d in docs]
corpus_emb = emb_model.encode(corpus, normalize_embeddings=True, convert_to_numpy=True)
index = faiss.IndexFlatIP(corpus_emb.shape[1])
index.add(corpus_emb)

# 3) 사용자 질의 → 관련 맛집 Top-k 검색
def search(q, k=3):
    q_emb = emb_model.encode([q], normalize_embeddings=True, convert_to_numpy=True)
    sims, idx = index.search(q_emb, k)
    return [(docs[i], float(sims[0][j])) for j,i in enumerate(idx[0])]

# 4) LLM(Qwen2-7B-Instruct)로 최종 추천문 생성
device = "cuda" if torch.cuda.is_available() else "cpu"
tok = AutoTokenizer.from_pretrained("Qwen/Qwen2-7B-Instruct")  # Apache-2.0
model = AutoModelForCausalLM.from_pretrained(
    "Qwen/Qwen2-7B-Instruct",
    torch_dtype=torch.bfloat16 if device=="cuda" else torch.float32,
    device_map="auto"
)

def recommend(user_query):
    top = search(user_query, k=3)
    context_lines = []
    for d,score in top:
        context_lines.append(f"- {d['name']} ({d['area']}, {d['price']}): {', '.join(d['tags'])} — {d['desc']}")
    context = "\n".join(context_lines)

    system = (
      "너는 한국어 미식 큐레이터야. 아래 후보 중에서 사용자 취향에 맞는 2곳을 선정하고, "
      "선정 이유(맛/분위기/가격/동선 등)를 구체적으로 3~4문장으로 설명해."
      "마지막에 한 줄 요약 TIP도 붙여줘."
    )
    prompt = (
      f"[시스템]\n{system}\n\n"
      f"[후보 맛집]\n{context}\n\n"
      f"[사용자 요청]\n{user_query}\n\n"
      "형식: \n1) 추천 리스트(이름 - 한줄소개)\n2) 상세 설명\n3) TIP"
    )

    inputs = tok(prompt, return_tensors="pt").to(model.device)
    out = model.generate(**inputs, max_new_tokens=500, temperature=0.7)
    return tok.decode(out[0], skip_special_tokens=True)[len(prompt):]

# 사용 예시
print(recommend("성수에서 데이트하고 파스타/와인 먹고 싶어. 분위기 좋은 곳 찾아줘"))


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]



1) 성수 파스타 - 파스타와 와인을 즐길 수 있는 분위기 좋은 데이트 장소
2) 생면 파스타를 제공하며, 자연스러운 와인 목록이 있어 데이트에 적합하다. 또한, 성수 위치 때문에 편리한 동선이 가능하다. 가격은 ₩₩₩으로 중간 정도이다.
3) TIP: 성수 파스타에서는 파스타와 와인을 동시에 즐길 수 있어 데이트 중에 서로의 취향을 확인하는 데 좋을 것 같다.

2) 을지로 골뱅이집 - 골뱅이와 소주로 이뤄진 특별한 데이트 장소
2) 골뱅이무침과 오징어볶음이 유명한 골뱅이집으로, 매콤한 맛과 함께 골뱅이를 즐길 수 있다. 또한, 노포 느낌의 분위기가 데이트에 적합하다. 가격은 ₩₩으로 저렴한 편이다.
3) TIP: 을지로 골뱅이집에서는 골뱅이를 주제로 한 특별한 데이트를 즐길 수 있다. 매콤한 맛과 함께 노포 느낌의 분위기를 즐길 수 있으니 추천한다.

요약 TIP: 데이트 중에는 성수 파스타나 을지로 골뱅이집 중에서 선택해보세요. 성수 파스타는 파스타와 와인을 즐길 수 있는 분위기 좋은 데이트 장소이며, 을지로 골뱅이집은 골뱅이와 소주로 이뤄진 특별한 데이트 장소입니다.


In [ ]:
# 데이터 로드
print("📊 데이터 로드 중...")

# Google Drive에서 데이터 다운로드 및 로드
data_paths = ensure_data_files()

# 각 데이터 파일 로드
diner_df = pd.read_csv('../data/diner.csv')
review_df = pd.read_csv('../data/review.csv')
menu_df = pd.read_csv('../data/dinr_menu.csv')
category_df = pd.read_csv('../data/diner_category_raw.csv')

print(f"✅ 식당 데이터: {diner_df.shape}")
print(f"✅ 리뷰 데이터: {review_df.shape}")
print(f"✅ 메뉴 데이터: {menu_df.shape}")
print(f"✅ 카테고리 데이터: {category_df.shape}")

# 컬럼 정보 확인
print("\n📋 데이터 컬럼 정보:")
print(f"식당: {diner_df.columns.tolist()}")
print(f"카테고리: {category_df.columns.tolist()}")
print(f"메뉴: {menu_df.columns.tolist()}")
print(f"리뷰: {review_df.columns.tolist()}")


In [ ]:
## 9. BGE-M3 임베딩과 Qwen2-7B-Instruct를 활용한 맛집 추천 시스템


In [ ]:
# BGE-M3와 Qwen2-7B-Instruct를 위한 추가 패키지 설치
%pip install "transformers>=4.40.0" "sentence-transformers>=2.7.0" "accelerate" "torch"

print("✅ BGE-M3 및 Qwen2-7B 관련 패키지 설치 완료")


In [13]:
# BGE-M3 임베딩 모델과 Qwen2-7B-Instruct 모델 로드
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import faiss
import numpy as np
from typing import List, Tuple, Dict
import warnings
warnings.filterwarnings("ignore")

print("🔧 모델 로드 시작...")

# 1. BGE-M3 한국어 임베딩 모델 로드
try:
    embedding_model = SentenceTransformer("BAAI/bge-m3")
    print("✅ BGE-M3 임베딩 모델 로드 완료")
except Exception as e:
    print(f"⚠️  BGE-M3 로드 실패, 한국어 BGE 모델로 대체: {e}")
    embedding_model = SentenceTransformer("upskyy/bge-m3-korean")

# 2. Qwen2-7B-Instruct 모델 로드
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"🖥️  사용 디바이스: {device}")

try:
    tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2-7B-Instruct")
    
    # 메모리 효율성을 위해 4bit 양자화 사용
    model = AutoModelForCausalLM.from_pretrained(
        "Qwen/Qwen2-7B-Instruct",
        torch_dtype=torch.bfloat16 if device == "cuda" else torch.float32,
        device_map="auto" if device == "cuda" else None,
        trust_remote_code=True
    )
    
    print("✅ Qwen2-7B-Instruct 모델 로드 완료")
    qwen_available = True
    
except Exception as e:
    print(f"⚠️  Qwen2-7B 로드 실패 - 경량화 모델로 대체: {e}")
    try:
        # 더 작은 모델로 대체
        tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2-1.5B-Instruct")
        model = AutoModelForCausalLM.from_pretrained(
            "Qwen/Qwen2-1.5B-Instruct",
            torch_dtype=torch.float32,
            device_map="auto" if device == "cuda" else None,
            trust_remote_code=True
        )
        print("✅ Qwen2-1.5B-Instruct 모델 로드 완료 (경량화 버전)")
        qwen_available = True
    except Exception as e2:
        print(f"❌ Qwen 모델 로드 완전 실패: {e2}")
        qwen_available = False
        tokenizer = None
        model = None

print("🚀 모델 초기화 완료!")


🔧 모델 로드 시작...
✅ BGE-M3 임베딩 모델 로드 완료
🖥️  사용 디바이스: cuda


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

✅ Qwen2-7B-Instruct 모델 로드 완료
🚀 모델 초기화 완료!


## 새로 생성한 Restaurant Documents으로 추천


In [ ]:
# 개선된 restaurant_docs 생성 함수

def create_improved_restaurant_docs(max_restaurants=200, min_reviews=5, region_filter='강남구'):
    """
    개선된 restaurant_docs 생성 함수
    
    Args:
        max_restaurants (int): 최대 식당 수 (기본값: 200)
        min_reviews (int): 최소 리뷰 수 (기본값: 5)
        region_filter (str): 지역 필터 (기본값: '강남구')
    
    Returns:
        List[Document]: 처리된 식당 Document 리스트
    """
    print(f"🔄 개선된 식당 데이터 처리 시작 ({region_filter} 지역)")
    print(f"📊 설정: 최대 {max_restaurants}개, 최소 리뷰수 {min_reviews}개")
    
    try:
        # 1. 기본 데이터 검증
        required_dfs = ['diner_df', 'review_df', 'menu_df', 'category_df']
        for df_name in required_dfs:
            if df_name not in globals() or globals()[df_name] is None:
                raise ValueError(f"{df_name}이 로드되지 않았습니다. 데이터를 먼저 로드해주세요.")
        
        # 2. 강남구 식당 필터링 (더 정확한 필터링)
        print("🔍 지역 필터링 중...")
        base_df = diner_df.copy()
        
        # 주소 기반 필터링 (도로명주소와 지번주소 모두 확인)
        region_mask = (
            base_df['diner_road_address'].fillna('').str.contains(region_filter, na=False) |
            base_df['diner_num_address'].fillna('').str.contains(region_filter, na=False)
        )
        base_df = base_df[region_mask]
        
        if base_df.empty:
            raise ValueError(f"{region_filter} 지역 데이터가 없습니다.")
        
        print(f"🏪 지역 필터링 결과: {len(base_df)}개 식당 발견")
        
        # 3. 리뷰 수 필터링 및 정리
        print("📝 리뷰 데이터 처리 중...")
        base_df['diner_review_cnt'] = pd.to_numeric(base_df['diner_review_cnt'], errors='coerce').fillna(0).astype(int)
        base_df['diner_review_avg'] = pd.to_numeric(base_df['diner_review_avg'], errors='coerce').fillna(0.0)
        
        # 최소 리뷰수 필터링
        base_df = base_df[base_df['diner_review_cnt'] >= min_reviews]
        print(f"📊 최소 리뷰수 {min_reviews}개 이상: {len(base_df)}개 식당")
        
        # 4. 카테고리 정보 병합 (더 정확한 매핑)
        print("🏷️  카테고리 정보 병합 중...")
        category_subset = category_df[category_df['diner_idx'].isin(base_df['diner_idx'])].copy()
        
        # 카테고리 정보를 더 풍부하게 결합
        category_grouped = category_subset.groupby('diner_idx').agg({
            'diner_category_large': 'first',
            'diner_category_middle': 'first', 
            'diner_category_small': 'first',
            'diner_category_detail': lambda x: ', '.join(x.dropna().unique()[:3])  # 최대 3개까지
        }).reset_index()
        
        base_df = base_df.merge(category_grouped, on='diner_idx', how='left')
        
        # 5. 메뉴 정보 개선된 집계
        print("🍽️  메뉴 정보 집계 중...")
        menu_subset = menu_df[menu_df['diner_idx'].isin(base_df['diner_idx'])].copy()
        menu_grouped = menu_subset.groupby('diner_idx')['diner_menu_name'].apply(
            lambda x: ' | '.join(x.dropna().unique()[:8])  # 최대 8개 메뉴
        ).reset_index()
        menu_grouped.columns = ['diner_idx', 'menu_list']
        base_df = base_df.merge(menu_grouped, on='diner_idx', how='left')
        
        # 6. 리뷰 정보 개선된 집계 (더 대표적인 리뷰 선택)
        print("💬 리뷰 정보 집계 중...")
        review_subset = review_df[review_df['diner_idx'].isin(base_df['diner_idx'])].copy()
        
        # 리뷰 정리: 날짜순 정렬 후 대표 리뷰 선택
        review_subset['reviewer_review_date'] = pd.to_datetime(review_subset['reviewer_review_date'], errors='coerce')
        review_subset = review_subset.dropna(subset=['reviewer_review'])
        
        # 각 식당별로 최신 리뷰와 높은 평점 리뷰를 혼합해서 선택
        def get_representative_reviews(group):
            # 최신 리뷰 3개와 높은 평점 리뷰 2개를 결합
            group = group.sort_values('reviewer_review_date', ascending=False)
            recent_reviews = group.head(3)['reviewer_review']
            
            high_score_reviews = group[group['reviewer_review_score'] >= 4].head(2)['reviewer_review']
            
            all_reviews = pd.concat([recent_reviews, high_score_reviews]).drop_duplicates()
            return '\n---\n'.join(all_reviews.dropna()[:5])  # 최대 5개
        
        review_grouped = review_subset.groupby('diner_idx').apply(get_representative_reviews).reset_index()
        review_grouped.columns = ['diner_idx', 'representative_reviews']
        base_df = base_df.merge(review_grouped, on='diner_idx', how='left')
        
        # 7. 상위 식당 선택 (베이지안 점수 고려)
        print("⭐ 상위 식당 선택 중...")
        if 'bayesian_score' in base_df.columns:
            # 베이지안 점수가 있으면 이를 우선 고려
            base_df = base_df.sort_values(['bayesian_score', 'diner_review_cnt'], ascending=[False, False])
        else:
            # 베이지안 점수가 없으면 리뷰수와 평점을 조합한 점수 계산
            base_df['calculated_score'] = (base_df['diner_review_avg'] * 0.7 + 
                                          np.log1p(base_df['diner_review_cnt']) * 0.3)
            base_df = base_df.sort_values('calculated_score', ascending=False)
        
        # 최대 개수만큼 선택
        if len(base_df) > max_restaurants:
            base_df = base_df.head(max_restaurants)
            print(f"🔝 상위 {max_restaurants}개 식당 선택")
        
        # 8. Document 생성
        print(f"📄 {len(base_df)}개 식당 Document 생성 중...")
        documents = []
        
        for idx, row in base_df.iterrows():
            # 더 풍부한 콘텐츠 생성
            content = f"""
식당명: {row.get('diner_name', '')}
카테고리: {row.get('diner_category_large', '')} > {row.get('diner_category_middle', '')} > {row.get('diner_category_small', '')}
세부카테고리: {row.get('diner_category_detail', '')}
태그: {row.get('diner_tag', '')}
메뉴: {row.get('menu_list', '')}
평점: {row.get('diner_review_avg', 0)}/5.0 (리뷰 {row.get('diner_review_cnt', 0)}개)
지역: {region_filter}
주소: {row.get('diner_road_address', '')}
영업시간: {row.get('diner_open_time', '')}

대표 리뷰:
{row.get('representative_reviews', '리뷰 없음')}
            """.strip()
            
            # 더 완전한 메타데이터
            metadata = {
                'diner_id': str(row.get('diner_idx', '')),
                'diner_name': str(row.get('diner_name', '')),
                'address': str(row.get('diner_road_address', '')),
                'region': region_filter,
                'category': str(row.get('diner_category_large', '')),
                'category_detail': str(row.get('diner_category_detail', '')),
                'tag': str(row.get('diner_tag', '')),
                'menu': str(row.get('menu_list', '')),
                'reviews': str(row.get('representative_reviews', '')),
                'rating': float(row.get('diner_review_avg', 0)),
                'review_count': int(row.get('diner_review_cnt', 0)),
                'opening_hours': str(row.get('diner_open_time', '')),
                'bayesian_score': float(row.get('bayesian_score', 0)) if 'bayesian_score' in row else 0.0
            }
            
            documents.append(Document(page_content=content, metadata=metadata))
        
        print(f"✅ {region_filter} 지역 {len(documents)}개 개선된 Document 생성 완료!")
        print(f"📈 평점 범위: {min([doc.metadata['rating'] for doc in documents]):.1f} ~ {max([doc.metadata['rating'] for doc in documents]):.1f}")
        print(f"📊 리뷰수 범위: {min([doc.metadata['review_count'] for doc in documents])} ~ {max([doc.metadata['review_count'] for doc in documents])}개")
        
        return documents
        
    except Exception as e:
        print(f"❌ 오류 발생: {e}")
        print("🔧 데이터를 확인하고 다시 시도해주세요.")
        return []

# 함수 정의 완료
print("✅ 개선된 restaurant_docs 생성 함수 준비 완료!")


✅ 개선된 restaurant_docs 생성 함수 준비 완료!


In [ ]:
# 새로운 개선된 restaurant_docs 생성 실행

print("🚀 새로운 restaurant_docs 생성 시작!")
print("="*60)

# 다양한 설정으로 생성 가능
# 옵션 1: 기본 설정 (강남구, 200개, 최소 리뷰 5개)
new_restaurant_docs = create_improved_restaurant_docs(
    max_restaurants=200, 
    min_reviews=5, 
    region_filter='강남구'
)

# 생성 결과 확인
if new_restaurant_docs:
    print("\n🎉 새로운 restaurant_docs 생성 완료!")
    print(f"📊 총 {len(new_restaurant_docs)}개 식당 Document")
    
    # 샘플 데이터 확인
    print("\n📋 샘플 데이터 (첫 번째 식당):")
    print("-" * 50)
    sample_doc = new_restaurant_docs[0]
    print(f"식당명: {sample_doc.metadata['diner_name']}")
    print(f"카테고리: {sample_doc.metadata['category']}")
    print(f"평점: {sample_doc.metadata['rating']}/5.0 ({sample_doc.metadata['review_count']}개 리뷰)")
    print(f"주소: {sample_doc.metadata['address']}")
    print(f"메뉴: {sample_doc.metadata['menu'][:100]}...")
    
    # 카테고리 분포 확인
    print("\n📊 카테고리 분포:")
    categories = [doc.metadata['category'] for doc in new_restaurant_docs if doc.metadata['category']]
    category_counts = pd.Series(categories).value_counts().head(10)
    for category, count in category_counts.items():
        print(f"   {category}: {count}개")
    
    # 평점 분포 확인
    ratings = [doc.metadata['rating'] for doc in new_restaurant_docs]
    print(f"\n⭐ 평점 분포:")
    print(f"   평균 평점: {np.mean(ratings):.2f}")
    print(f"   평점 범위: {min(ratings):.1f} ~ {max(ratings):.1f}")
    print(f"   4.0 이상 식당: {len([r for r in ratings if r >= 4.0])}개")
    
    # 기존 restaurant_docs와 비교
    if 'restaurant_docs' in globals() and restaurant_docs:
        print(f"\n🔄 기존 대비 개선사항:")
        print(f"   기존: {len(restaurant_docs)}개 → 신규: {len(new_restaurant_docs)}개")
        print(f"   데이터 품질: 더 풍부한 메타데이터, 대표적인 리뷰, 세부 카테고리 정보")
        
        # 기존 데이터를 새 데이터로 교체할지 선택
        replace_old = True  # 기본값으로 교체
        if replace_old:
            restaurant_docs = new_restaurant_docs
            print("✅ 기존 restaurant_docs를 새로운 데이터로 교체했습니다!")
        else:
            print("ℹ️  기존 restaurant_docs는 유지하고, new_restaurant_docs 변수에 저장했습니다.")
    else:
        # 기존 데이터가 없으면 자동으로 설정
        restaurant_docs = new_restaurant_docs
        print("✅ restaurant_docs 변수에 새로운 데이터를 설정했습니다!")
    
else:
    print("❌ restaurant_docs 생성에 실패했습니다.")
    print("🔧 데이터 로드 상태를 확인해주세요.")

print("\n" + "="*60)
print("🔍 생성된 restaurant_docs를 BGE-M3 임베딩에 사용할 수 있습니다!")


🚀 새로운 restaurant_docs 생성 시작!
🔄 개선된 식당 데이터 처리 시작 (강남구 지역)
📊 설정: 최대 200개, 최소 리뷰수 5개
🔍 지역 필터링 중...


🏪 지역 필터링 결과: 10555개 식당 발견
📝 리뷰 데이터 처리 중...
📊 최소 리뷰수 5개 이상: 5748개 식당
🏷️  카테고리 정보 병합 중...
🍽️  메뉴 정보 집계 중...
💬 리뷰 정보 집계 중...
⭐ 상위 식당 선택 중...
🔝 상위 200개 식당 선택
📄 200개 식당 Document 생성 중...
✅ 강남구 지역 200개 개선된 Document 생성 완료!
📈 평점 범위: 2.7 ~ 5.0
📊 리뷰수 범위: 5 ~ 975개

🎉 새로운 restaurant_docs 생성 완료!
📊 총 200개 식당 Document

📋 샘플 데이터 (첫 번째 식당):
--------------------------------------------------
식당명: 지미존스 강남역점
카테고리: 패스트푸드
평점: 4.6/5.0 (22개 리뷰)
주소: 서울 강남구 강남대로 358
메뉴: ['빌리 클럽', '미트볼 파마산', '얼티밋 포커', '스윗 칠리 치킨', '터키톰', '페페']...

📊 카테고리 분포:
   한식: 62개
   중식: 23개
   일식: 21개
   양식: 21개
   술집: 17개
   카페: 10개
   간식: 10개
   퓨전요리: 7개
   패스트푸드: 6개
   분식: 6개

⭐ 평점 분포:
   평균 평점: 3.76
   평점 범위: 2.7 ~ 5.0
   4.0 이상 식당: 89개

🔄 기존 대비 개선사항:
   기존: 200개 → 신규: 200개
   데이터 품질: 더 풍부한 메타데이터, 대표적인 리뷰, 세부 카테고리 정보
✅ 기존 restaurant_docs를 새로운 데이터로 교체했습니다!

🔍 생성된 restaurant_docs를 BGE-M3 임베딩에 사용할 수 있습니다!


/var/tmp/ipykernel_179573/1960898824.py:92: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  review_grouped = review_subset.groupby('diner_idx').apply(get_representative_reviews).reset_index()


In [ ]:
print(restaurant_docs[0])

page_content='식당명: 지미존스 강남역점
카테고리: 패스트푸드 > 샌드위치 > None
세부카테고리: 
태그: ['카카오페이', '지미존스', '샌드위치']
메뉴: ['빌리 클럽', '미트볼 파마산', '얼티밋 포커', '스윗 칠리 치킨', '터키톰', '페페']
평점: 4.6/5.0 (리뷰 22개)
지역: 강남구
주소: 서울 강남구 강남대로 358
전화번호: 02-2051-2024
영업시간: nan

대표 리뷰:
맛있고 서브웨이보다 고급진 맛이에요
근데 부탁 하나만 할께요
제가 주차 등록 해달라고 하니까 
요리용 라텍스 장갑 끼시고 마우스랑 키보드 만지시면서 주차등록 하시고 다시 그대로 샌드위치 만드시던데요…. 정말 그 장갑으로 다시 요리하신다고요? 라고 말하고 싶었는데 참았어요.
본사 지침은 분명 그게 아닐텐데요?
많이 아쉬운 장면이었습니다…..
장갑 그렇게 아끼시지 않으셨음 좋겠어요
더보기
---
맛있음
---
이 일대에서 맛과 가격이 일치하는 식당 중 하나. 빵 로스트한거 쫀쫀하니 맛남. 그냥 이상한 데에 돈쓰기 싫으면 아묻따 지미존스감.' metadata={'diner_id': '1357207453', 'diner_name': '지미존스 강남역점', 'address': '서울 강남구 강남대로 358', 'region': '강남구', 'category': '패스트푸드', 'category_detail': '', 'tag': "['카카오페이', '지미존스', '샌드위치']", 'menu': "['빌리 클럽', '미트볼 파마산', '얼티밋 포커', '스윗 칠리 치킨', '터키톰', '페페']", 'reviews': '맛있고 서브웨이보다 고급진 맛이에요\n근데 부탁 하나만 할께요\n제가 주차 등록 해달라고 하니까 \n요리용 라텍스 장갑 끼시고 마우스랑 키보드 만지시면서 주차등록 하시고 다시 그대로 샌드위치 만드시던데요…. 정말 그 장갑으로 다시 요리하신다고요? 라고 말하고 싶었는데 참았어요.\n본사 지침은 분명 그게 아닐텐데요?\n많이 아쉬운 장면

In [10]:
# 다른 지역이나 설정으로 restaurant_docs 생성하기 (옵션)

def create_custom_restaurant_docs():
    """사용자 맞춤형 restaurant_docs 생성"""
    
    print("🎯 맞춤형 restaurant_docs 생성 옵션들:")
    print("="*50)
    
    # 옵션 1: 더 많은 데이터 (300개, 최소 리뷰 3개)
    print("📊 옵션 1: 더 많은 강남구 식당 (300개, 최소 리뷰 3개)")
    # large_docs = create_improved_restaurant_docs(max_restaurants=300, min_reviews=3)
    
    # 옵션 2: 고급 식당만 (100개, 최소 리뷰 20개, 평점 4.0 이상)
    print("⭐ 옵션 2: 고급 맛집만 (100개, 최소 리뷰 20개)")
    # premium_docs = create_improved_restaurant_docs(max_restaurants=100, min_reviews=20)
    
    # 옵션 3: 다른 지역 (예: 홍대, 강남, 마포구 등)
    print("🗺️  옵션 3: 다른 지역들")
    available_regions = ['강남구', '서초구', '마포구', '중구', '종로구', '용산구']
    print(f"   사용 가능한 지역: {', '.join(available_regions)}")
    
    return available_regions

# 사용 가능한 옵션들 보여주기
regions = create_custom_restaurant_docs()

print(f"""
💡 사용 예시:

# 더 많은 데이터로 생성하기
more_docs = create_improved_restaurant_docs(max_restaurants=300, min_reviews=3)

# 고급 맛집만 생성하기  
premium_docs = create_improved_restaurant_docs(max_restaurants=100, min_reviews=20)

# 다른 지역으로 생성하기
hongdae_docs = create_improved_restaurant_docs(max_restaurants=200, min_reviews=5, region_filter='마포구')
seocho_docs = create_improved_restaurant_docs(max_restaurants=200, min_reviews=5, region_filter='서초구')

# 현재 생성된 restaurant_docs 정보 확인하기
print(f"현재 restaurant_docs: {{len(restaurant_docs)}}개 식당")
""")

# 현재 데이터 상태 요약
if 'restaurant_docs' in globals() and restaurant_docs:
    print("\n📈 현재 restaurant_docs 상태:")
    print(f"   식당 수: {len(restaurant_docs)}개")
    categories = [doc.metadata.get('category', '') for doc in restaurant_docs if doc.metadata.get('category')]
    if categories:
        top_category = pd.Series(categories).value_counts().index[0]
        print(f"   주요 카테고리: {top_category}")
    
    ratings = [doc.metadata.get('rating', 0) for doc in restaurant_docs]
    if ratings:
        print(f"   평균 평점: {np.mean(ratings):.2f}")
        print(f"   고평점 식당 (4.0+): {len([r for r in ratings if r >= 4.0])}개")
    
    print("✅ BGE-M3 임베딩 시스템에서 사용할 준비가 완료되었습니다!")
else:
    print("⚠️  restaurant_docs가 아직 생성되지 않았습니다. 위의 셀을 먼저 실행해주세요.")


🎯 맞춤형 restaurant_docs 생성 옵션들:
📊 옵션 1: 더 많은 강남구 식당 (300개, 최소 리뷰 3개)
⭐ 옵션 2: 고급 맛집만 (100개, 최소 리뷰 20개)
🗺️  옵션 3: 다른 지역들
   사용 가능한 지역: 강남구, 서초구, 마포구, 중구, 종로구, 용산구

💡 사용 예시:

# 더 많은 데이터로 생성하기
more_docs = create_improved_restaurant_docs(max_restaurants=300, min_reviews=3)

# 고급 맛집만 생성하기  
premium_docs = create_improved_restaurant_docs(max_restaurants=100, min_reviews=20)

# 다른 지역으로 생성하기
hongdae_docs = create_improved_restaurant_docs(max_restaurants=200, min_reviews=5, region_filter='마포구')
seocho_docs = create_improved_restaurant_docs(max_restaurants=200, min_reviews=5, region_filter='서초구')

# 현재 생성된 restaurant_docs 정보 확인하기
print(f"현재 restaurant_docs: {len(restaurant_docs)}개 식당")


📈 현재 restaurant_docs 상태:
   식당 수: 200개
   주요 카테고리: 한식
   평균 평점: 3.76
   고평점 식당 (4.0+): 89개
✅ BGE-M3 임베딩 시스템에서 사용할 준비가 완료되었습니다!


In [11]:
# 커스텀 쿼리 테스트 (사용자가 직접 입력 가능)

def interactive_recommendation():
    """사용자가 직접 쿼리를 입력해서 추천받을 수 있는 함수"""
    print("🍽️  BGE-M3 + Qwen2-7B 맛집 추천 시스템")
    print("=" * 50)
    print("원하는 맛집을 자유롭게 설명해보세요!")
    print("예시: '강남구에서 친구들과 갈 만한 이탈리안 레스토랑'")
    print("     '혼밥하기 좋은 조용한 한식당'")
    print("     '데이트하기 좋은 분위기 있는 카페'")
    print("-" * 50)
    
    # 사용자 입력 받기 (Jupyter에서는 input() 사용)
    user_query = input("🎯 어떤 맛집을 찾고 계신가요? ")
    
    if not user_query.strip():
        print("❌ 쿼리가 입력되지 않았습니다.")
        return
    
    print(f"\n🔍 '{user_query}' 검색 중...")
    print("⏳ 잠시만 기다려주세요...\n")
    
    try:
        result = recommend_restaurants_bge_qwen(user_query)
        display_recommendation(result)
        
        print("\n💡 다른 추천을 원하시면 위 셀을 다시 실행하세요!")
        
    except Exception as e:
        print(f"❌ 추천 생성 중 오류가 발생했습니다: {e}")
        print("🔧 시스템 상태를 확인하고 다시 시도해주세요.")

# 자동 실행하지 않고 사용자가 필요할 때 실행할 수 있도록 함수만 정의
print("💬 사용자 맞춤 추천을 받으려면 다음을 실행하세요:")
print("interactive_recommendation()")
print("\n또는 직접 함수를 호출하세요:")
print('recommend_restaurants_bge_qwen("원하는 쿼리 입력")')


💬 사용자 맞춤 추천을 받으려면 다음을 실행하세요:
interactive_recommendation()

또는 직접 함수를 호출하세요:
recommend_restaurants_bge_qwen("원하는 쿼리 입력")


In [14]:
# 강남구 맛집 데이터를 BGE-M3로 임베딩하여 FAISS 인덱스 생성
print("🍽️  강남구 맛집 데이터 임베딩 시작...")

def create_restaurant_corpus(docs):
    """Document 리스트를 임베딩용 텍스트 코퍼스로 변환"""
    corpus = []
    for doc in docs:
        meta = doc.metadata
        # 식당 정보를 구조화된 텍스트로 변환
        text = (f"{meta['diner_name']} | {meta['category']} | {meta['region']} | "
                f"평점:{meta['rating']} | 리뷰수:{meta['review_count']} | "
                f"태그:{meta['tag']} | 메뉴:{meta['menu'][:100]} | "
                f"리뷰샘플:{meta['reviews'][:200]}")
        corpus.append(text)
    return corpus

# 1. 기존 강남구 데이터 활용
if 'restaurant_docs' in globals() and restaurant_docs:
    print(f"📋 기존 강남구 데이터 활용: {len(restaurant_docs)}개 식당")
    
    # 2. 임베딩용 코퍼스 생성
    restaurant_corpus = create_restaurant_corpus(restaurant_docs)
    
    # 3. BGE-M3로 임베딩 생성
    print("🔄 BGE-M3 임베딩 생성 중...")
    try:
        restaurant_embeddings = embedding_model.encode(
            restaurant_corpus, 
            normalize_embeddings=True, 
            convert_to_numpy=True,
            show_progress_bar=True
        )
        
        # 4. FAISS 인덱스 생성 (Inner Product = Cosine Similarity with normalized vectors)
        dimension = restaurant_embeddings.shape[1]
        faiss_index = faiss.IndexFlatIP(dimension)
        faiss_index.add(restaurant_embeddings.astype('float32'))
        
        print(f"✅ FAISS 인덱스 생성 완료: {faiss_index.ntotal}개 벡터, {dimension}차원")
        embedding_success = True
        
    except Exception as e:
        print(f"❌ 임베딩 생성 실패: {e}")
        embedding_success = False
        faiss_index = None
        restaurant_embeddings = None
        
else:
    print("❌ 강남구 데이터(restaurant_docs)가 없습니다. 이전 셀을 먼저 실행하세요.")
    embedding_success = False

print("🔍 임베딩 기반 검색 시스템 준비 완료!")


🍽️  강남구 맛집 데이터 임베딩 시작...
📋 기존 강남구 데이터 활용: 200개 식당
🔄 BGE-M3 임베딩 생성 중...


Batches:   0%|          | 0/7 [00:00<?, ?it/s]

✅ FAISS 인덱스 생성 완료: 200개 벡터, 1024차원
🔍 임베딩 기반 검색 시스템 준비 완료!


In [15]:
# BGE-M3 검색과 Qwen2-7B 추천을 결합한 맛집 추천 시스템 구현

def search_similar_restaurants(query: str, top_k: int = 5) -> List[Tuple[Dict, float]]:
    """BGE-M3를 사용해 사용자 쿼리와 유사한 맛집을 검색"""
    if not embedding_success or faiss_index is None:
        return []
    
    # 쿼리 임베딩
    query_embedding = embedding_model.encode(
        [query], 
        normalize_embeddings=True, 
        convert_to_numpy=True
    )
    
    # FAISS로 유사도 검색
    similarities, indices = faiss_index.search(query_embedding.astype('float32'), top_k)
    
    # 결과 반환
    results = []
    for i, idx in enumerate(indices[0]):
        if idx < len(restaurant_docs):
            results.append((restaurant_docs[idx].metadata, float(similarities[0][i])))
    
    return results

def generate_recommendation_with_qwen(user_query: str, similar_restaurants: List[Tuple[Dict, float]]) -> str:
    """Qwen2-7B-Instruct를 사용해 개인화된 추천 문구 생성"""
    if not qwen_available or not similar_restaurants:
        return "추천 시스템을 사용할 수 없습니다."
    
    # 컨텍스트 생성 (상위 3개 식당만 사용)
    context_lines = []
    for i, (restaurant, score) in enumerate(similar_restaurants[:3], 1):
        line = (f"{i}. {restaurant['diner_name']} ({restaurant['category']}) "
                f"- 평점: {restaurant['rating']}, 리뷰수: {restaurant['review_count']}\n"
                f"   주소: {restaurant['address']}\n"
                f"   특징: {restaurant['tag']} | {restaurant['menu'][:50]}...")
        context_lines.append(line)
    
    context = "\n".join(context_lines)
    
    # 프롬프트 생성
    system_prompt = (
        "당신은 강남구 전문 맛집 추천 AI입니다. "
        "사용자의 요청에 맞는 맛집 2-3곳을 선별하여 추천하고, "
        "각 맛집의 특징과 추천 이유를 친근하고 구체적으로 설명해주세요."
    )
    
    user_prompt = (
        f"[사용자 요청]\n{user_query}\n\n"
        f"[후보 맛집 정보]\n{context}\n\n"
        "위 후보 중에서 사용자 요청에 가장 적합한 2-3곳을 선택하여 추천해주세요. "
        "각 맛집의 특징, 분위기, 추천 이유를 포함해서 설명해주세요."
    )
    
    # Qwen2 입력 형식에 맞게 메시지 구성
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}
    ]
    
    try:
        # 토크나이저로 입력 생성
        text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
        inputs = tokenizer(text, return_tensors="pt").to(model.device)
        
        # 생성
        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_new_tokens=400,
                temperature=0.7,
                do_sample=True,
                pad_token_id=tokenizer.eos_token_id
            )
        
        # 결과 디코딩
        response = tokenizer.decode(outputs[0], skip_special_tokens=True)
        
        # 응답에서 생성된 부분만 추출
        if "assistant" in response:
            response = response.split("assistant")[-1].strip()
        elif user_prompt in response:
            response = response.replace(user_prompt, "").strip()
        
        return response
        
    except Exception as e:
        print(f"⚠️  Qwen2 생성 중 오류: {e}")
        # 대체 응답 생성
        fallback_response = f"강남구에서 '{user_query}'에 맞는 맛집을 추천드립니다:\n\n"
        for i, (restaurant, score) in enumerate(similar_restaurants[:2], 1):
            fallback_response += (
                f"{i}. **{restaurant['diner_name']}** ({restaurant['category']})\n"
                f"   - 평점: {restaurant['rating']}/5.0 (리뷰 {restaurant['review_count']}개)\n"
                f"   - 주소: {restaurant['address']}\n"
                f"   - 유사도: {score:.2f}\n\n"
            )
        return fallback_response

def recommend_restaurants_bge_qwen(user_query: str) -> Dict:
    """BGE-M3 + Qwen2-7B를 결합한 최종 추천 함수"""
    print(f"🔍 검색 쿼리: '{user_query}'")
    
    # 1. BGE-M3로 유사한 맛집 검색
    similar_restaurants = search_similar_restaurants(user_query, top_k=5)
    
    if not similar_restaurants:
        return {"error": "검색 결과가 없습니다.", "restaurants": [], "recommendation": ""}
    
    print(f"📋 검색된 맛집: {len(similar_restaurants)}개")
    
    # 2. Qwen2-7B로 개인화된 추천 생성
    recommendation_text = generate_recommendation_with_qwen(user_query, similar_restaurants)
    
    # 3. 결과 반환
    return {
        "query": user_query,
        "restaurants": [
            {
                "name": restaurant["diner_name"],
                "category": restaurant["category"],
                "rating": restaurant["rating"],
                "review_count": restaurant["review_count"],
                "address": restaurant["address"],
                "similarity_score": round(score, 3)
            }
            for restaurant, score in similar_restaurants
        ],
        "recommendation": recommendation_text
    }

print("🚀 BGE-M3 + Qwen2-7B 추천 시스템 구현 완료!")


🚀 BGE-M3 + Qwen2-7B 추천 시스템 구현 완료!


In [16]:
# BGE-M3 + Qwen2-7B 맛집 추천 시스템 테스트

def display_recommendation(result):
    """추천 결과를 보기 좋게 출력"""
    if "error" in result:
        print(f"❌ 오류: {result['error']}")
        return
    
    print(f"🔍 검색 쿼리: '{result['query']}'")
    print("="*80)
    
    print("📋 검색된 맛집 목록:")
    for i, restaurant in enumerate(result['restaurants'], 1):
        print(f"  {i}. {restaurant['name']} ({restaurant['category']})")
        print(f"     평점: {restaurant['rating']}/5.0 | 리뷰: {restaurant['review_count']}개 | 유사도: {restaurant['similarity_score']}")
        print(f"     주소: {restaurant['address']}")
        print()
    
    print("🤖 AI 추천 결과:")
    print("-" * 60)
    print(result['recommendation'])
    print("=" * 80)

# 다양한 테스트 쿼리
test_queries = [
    "강남구에서 혼밥하기 좋은 스시집 추천해줘",
    "회식하기 좋은 한식당이 어디 있을까?", 
    "데이트하기 좋은 양식 레스토랑 찾아줘",
    "가성비 좋은 점심 식사 할 곳 추천",
    "분위기 좋은 카페나 브런치 맛집"
]

print("🧪 BGE-M3 + Qwen2-7B 맛집 추천 시스템 테스트 시작!")
print("\n")

# 각 테스트 쿼리 실행
for i, query in enumerate(test_queries, 1):
    print(f"📝 테스트 {i}/5")
    try:
        result = recommend_restaurants_bge_qwen(query)
        display_recommendation(result)
        print("\n" + "🔄 다음 테스트로 이동...\n")
        
    except Exception as e:
        print(f"❌ 테스트 {i} 실패: {e}")
        print("🔄 다음 테스트로 이동...\n")

print("🎉 모든 테스트 완료!")
print("\n📊 시스템 요약:")
print(f"  - 임베딩 모델: BGE-M3 ({'✅ 정상' if embedding_success else '❌ 실패'})")  
print(f"  - 생성 모델: Qwen2-7B-Instruct ({'✅ 정상' if qwen_available else '❌ 실패'})")
print(f"  - 강남구 맛집 데이터: {len(restaurant_docs) if 'restaurant_docs' in globals() else 0}개")
print(f"  - FAISS 인덱스: {faiss_index.ntotal if embedding_success and faiss_index else 0}개 벡터")


🧪 BGE-M3 + Qwen2-7B 맛집 추천 시스템 테스트 시작!


📝 테스트 1/5
🔍 검색 쿼리: '강남구에서 혼밥하기 좋은 스시집 추천해줘'
📋 검색된 맛집: 5개
🔍 검색 쿼리: '강남구에서 혼밥하기 좋은 스시집 추천해줘'
📋 검색된 맛집 목록:
  1. 김태완스시 대치본점 (일식)
     평점: 3.3/5.0 | 리뷰: 975개 | 유사도: 0.604
     주소: 서울 강남구 선릉로64길 17

  2. 스시이안앤 강남역점 (일식)
     평점: 2.7/5.0 | 리뷰: 25개 | 유사도: 0.588
     주소: 서울 강남구 테헤란로1길 25

  3. 샤브로21 강남점 (샤브샤브)
     평점: 3.6/5.0 | 리뷰: 36개 | 유사도: 0.586
     주소: 서울 강남구 강남대로102길 13-5

  4. 서울하노이 (아시아음식)
     평점: 2.7/5.0 | 리뷰: 10개 | 유사도: 0.583
     주소: 서울 강남구 언주로98길 13

  5. 판교집 학동직영점 (한식)
     평점: 3.8/5.0 | 리뷰: 8개 | 유사도: 0.579
     주소: 서울 강남구 학동로 158

🤖 AI 추천 결과:
------------------------------------------------------------
1. **김태완스시 대치본점**
   
   이 맛집은 강남구 내에 위치한 고급스러운 스타일의 스시집으로, 특히 점심 시간에는 빠르고 효율적인 서비스를 제공합니다. '날치알군함(2ps)', '두가지초밥4', '세트메뉴1' 등 다양한 메뉴를 제공하며, 특히 '메밀(여름/4월~9월)'이라는 시즌별 메뉴는 시원한 여름철에 더욱 추천됩니다. '점심식사'라는 특징 때문에 혼밥에 적합하며, 평균적인 가격대는 고급스럽지만, 비교적 많은 리뷰 수와 평점을 보아 가격 대비 만족도가 높은 곳으로 추천드립니다.

2. **스시이안앤 강남역점**
   
   이 맛집은 회전초밥을 특화한 곳으로, 한 접시당 비싸지 않은 가격으로 